## Tutorial on how to create, invoke, and display a Constructive Solid Geometry (CSG) Tree Graph
https://en.wikipedia.org/wiki/Constructive_solid_geometry

In [ ]:
# Import TopologicPy modules. This is not needed on other computers
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

### Import the needed libraries

In [ ]:
from topologicpy.Cell import Cell
from topologicpy.Topology import Topology
from topologicpy.Cluster import Cluster
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
from topologicpy.Helper import Helper
from topologicpy.CSG import CSG
print("This jupyter notebook requires topologicpy v.0.8.23 or newer")
print(Helper.Version())

### Initialize the CSG Tree Graph

In [ ]:
csg_graph = CSG.Init()
print("CSG Graph:", csg_graph)

### Create Topology Vertices (i.e. Primitives)
In this tutorial, two cylinders will be unioned to form a cross-shape and then they will be subtracted from a cube.

Each topology is represented by a topology vertex in the CSG graph.

Topology vertices should only be connected to one and only one operation vertex (see below).

In [ ]:
# Create geometry vertices
cylinder_1 = Cell.Cylinder(radius=0.4, height=2)
v_cylinder_1 = CSG.AddTopologyVertex(csg_graph, cylinder_1, matrix=None, mantissa=6, tolerance=0.0001)

cylinder_2 = Cell.Cylinder(radius=0.4, height=2, direction=[1,0,0])
v_cylinder_2 = CSG.AddTopologyVertex(csg_graph, cylinder_2, matrix=None, mantissa=6, tolerance=0.0001)

cube = Cell.Cube(size=1)
v_cube = CSG.AddTopologyVertex(csg_graph, cube, matrix=None, mantissa=6, tolerance=0.0001)
print("Done creating the topology vertices")

### Create the Operations Vertices (i.e. Boolean operations)

Boolean operations are also represented as vertices in the CSG graph. Each operation vertex should be connected to two and only two other vertices. These child vertices can be either a topology vertex or another operation vertex.

In [ ]:
# Create union operation vertex
v_union = CSG.AddOperationVertex(csg_graph, "UNION", a=v_cylinder_1, b=v_cylinder_2, tolerance=0.0001)

# Create difference operation vertex (root)
v_diff =CSG.AddOperationVertex(csg_graph, "DIFFERENCE", a=v_cube, b=v_union, tolerance=0.0001)

### Create the graph edges by "connecting" the previously created vertices.

In [ ]:
# Create Edges from children to parent
csg_graph = CSG.Connect(csg_graph, v_cylinder_1, v_union)
csg_graph = CSG.Connect(csg_graph, v_cylinder_2, v_union)
csg_graph = CSG.Connect(csg_graph, v_cube, v_diff)
csg_graph = CSG.Connect(csg_graph, v_union, v_diff)

### Reshape the graph as a tree with the last operation (e.g. v_diff) as its root

In [ ]:
csg_graph = Graph.Reshape(csg_graph, shape="Tree 2D", rootVertex=v_diff)

### Invoke the graph to solve for the final shape

In [ ]:
final_geometry = CSG.Invoke(csg_graph)
Topology.Show(final_geometry)

### Show the graph and the topologies placed at each graph vertex

In [ ]:
topologies = CSG.Topologies(csg_graph, xOffset=0, yOffset=0, zOffset=-0.4, scale=0.4)
data_graph = Plotly.DataByGraph(csg_graph, vertexSize=8, edgeWidth=4, showVertexLabel=True, vertexLabelKey="operation")
data_top = Plotly.DataByTopology(Cluster.ByTopologies(topologies))
figure = Plotly.FigureByData(data_graph+data_top)
Plotly.Show(figure)